In [1]:
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.style.use('seaborn-darkgrid')

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# 1. Snake

## 1.1. Multi Layer Perceptron

In [2]:
import numpy as np
from pysnake import Game, Snake

In [3]:
game = Game((15, 15))
snake = Snake(game)

In [4]:
print(snake.nn_layers_dimension)

[32, 32, 32, 4]


### 1.1.1. Convert Vision to Array

In [5]:
X = snake.compute_input()
print(snake.full_vision)
print(snake.direction)
print(snake.tail_direction)
print(X)

Direction.UP
Direction.UP
[[0.38074981]
 [0.        ]
 [0.        ]
 [0.53846154]
 [0.        ]
 [0.        ]
 [0.38074981]
 [0.        ]
 [0.        ]
 [0.53846154]
 [0.        ]
 [0.        ]
 [0.38074981]
 [0.05439283]
 [0.        ]
 [0.53846154]
 [0.        ]
 [0.        ]
 [0.38074981]
 [0.        ]
 [0.        ]
 [0.53846154]
 [0.        ]
 [0.        ]
 [1.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [1.        ]
 [0.        ]
 [0.        ]
 [0.        ]]


In [6]:
Y_hat = snake.compute_output(X)
print(Y_hat)
print(snake.next_direction())

[[0.08978496]
 [0.0646567 ]
 [0.72156848]
 [0.12398987]]
Direction.DOWN


# 2. Genetic Algorithm

In [7]:
def play_multiple_games(game, list_snakes):
    for snake in list_snakes:
        # Make sure there are no snakes before running
        game.restart(snake)
        game.run()

In [8]:
from pysnake.enum import Direction
from pysnake import Game, Snake
from pysnake.ui import WindowGame

from pysnake.gen import Population, Individual, Chromosome

from math import sqrt

NUM_INDIVIDUALS = 150
NUM_GENERATIONS = 300
BOARD_SIZE = (15, 15)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [11]:
def genetic_algo():
    fitness = []  # For tracking average fitness over generation
    
    # Create and initialize the population
    game = Game(BOARD_SIZE)
    individuals = [Snake(game, id=i) for i in range(NUM_INDIVIDUALS)]
    pop = Population(individuals)
          
    for generation in range(NUM_GENERATIONS):
        
        # Play the game for each snake
        play_multiple_games(game, pop.individuals)
        
        percentage = generation * 100 / (NUM_GENERATIONS - 1)
        loading = "=" * round((percentage //2)) + " " * round((50 - percentage // 2))
        print("\rGeneration :{0:4d}/{1} | [{2}] | best fitness : {3:2.3E} | best score : {4:2d} | lifespan : {5:3d}".format(
                                                                    generation+1, 
                                                                    NUM_GENERATIONS, 
                                                                    loading, 
                                                                    pop.fittest.fitness,
                                                                    pop.fittest.score,
                                                                    pop.fittest.lifespan), 
              end="")
        next_pop = []  # For setting next population
        
        # Get best individuals from current pop
        best_from_pop = pop.select_elitism(50)
        next_pop.extend(best_from_pop)
        
        while len(next_pop) < NUM_INDIVIDUALS:
            #p1, p2 = pop.select_roulette_wheel(2)
            p1, p2 = pop.select_tournament(2, 100)
            mutation_rate = 0.05 / sqrt(generation + 1)
        
            # Create offpsring through crossover
            c1_chromosome, c2_chromosome = pop.crossover_simulated_binary(p1, p2, 1)
                        
            c1 = Snake(game, chromosomes=c1_chromosome)
            c2 = Snake(game, chromosomes=c2_chromosome)          

            # Mutate offspring
            c1.mutate(mutation_rate)
            c2.mutate(mutation_rate)

            # Add to next population
            next_pop.extend([c1, c2])
               
        # Track average fitness
        fitness.append(pop.mean_fitness)

        # Set the next generation
        pop.individuals = next_pop
        
    print("\n\nBest individual :\n")
    print(pop.fittest)
    
    plt.yscale('symlog')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.plot(range(len(fitness)), fitness, color='steelblue')
    plt.tight_layout()
    plt.show()
            
    return pop

In [12]:
pop = genetic_algo()

Generation :  39/300 | [======                                            ] | best fitness : 0.000E+00 | best score :  0 | lifespan :  12

KeyboardInterrupt: 

In [ ]:
print(pop.individuals[-1].nn_params)